In [1]:

import pandas as pd


In [2]:
df = pd.read_csv('Gemstone.csv',nrows=1000)

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df.drop(labels=['id'],axis=1)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
995,0.39,Ideal,J,VS2,61.4,53.8,4.69,4.74,2.90,731
996,1.00,Ideal,D,SI1,61.9,55.0,6.39,6.41,3.96,6040
997,0.71,Ideal,D,SI1,62.3,56.0,5.68,5.72,3.55,2732
998,0.54,Ideal,F,VS2,61.8,56.0,5.29,5.24,3.25,1586


In [5]:
#seggregate independent and dependent feature
X=df.drop(labels=['price'],axis=1)
Y=df['price']

In [6]:
Y

0      13619
1      13387
2       2772
3        666
4      14453
       ...  
995      731
996     6040
997     2732
998     1586
999      596
Name: price, Length: 1000, dtype: int64

In [7]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [8]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.impute import SimpleImputer #for handling missing values
from sklearn.preprocessing import StandardScaler #Handling Feature scaling
from sklearn.preprocessing import OrdinalEncoder #ordinal Encoding
##
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

 


In [10]:
#Numerical_pipeline
numerical_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
#Categorical_pipeline
categorical_pipeleine=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)
preprocessor=ColumnTransformer([
    ('num_pipeline',numerical_pipeline,numerical_cols),
    ('cat_pipeline',categorical_pipeleine,categorical_cols)
])

In [11]:
##Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [12]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [13]:
X_train.head()

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.143479,-0.979945,1.112105,-1.183119,-1.166083,-1.147671,-1.087336,-1.132275,1.523785,-0.586935
1,-0.208437,-0.979945,-0.245810,-0.640012,-1.138675,-1.120126,-1.146266,0.892227,-0.314210,1.972907
2,-0.062096,-0.958088,-0.245810,-0.640012,-1.083860,-1.065037,-1.087336,0.892227,0.298455,1.972907
3,-0.271155,-0.105691,-0.148816,-1.726226,0.094674,0.137750,0.105989,0.892227,0.298455,0.692986
4,0.969262,-0.608387,1.500080,-0.096906,-0.572248,-0.624321,-0.483307,-2.144526,0.911120,-0.586935


In [14]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [15]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [16]:
regression.coef_

array([ 1.76479341e+00,  6.92417763e+03,  5.14387799e+02, -2.27902660e+01,
        2.33074403e+03,  2.47965958e+03, -7.59749733e+03,  9.64830697e+01,
       -4.10749568e+02,  6.60253706e+02])

In [17]:
regression.intercept_

3861.435714285706

In [18]:
import numpy as np 
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square




In [19]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1002.0907670271617
MAE: 682.5756494337128
R2 score 94.47092101695621


Lasso
Model Training Performance
RMSE: 991.1733868270431
MAE: 685.6331641275424
R2 score 94.59073898882652


Ridge
Model Training Performance
RMSE: 993.8301964585971
MAE: 695.4555819492529
R2 score 94.56170140980966


Elasticnet
Model Training Performance
RMSE: 1605.9217313954302
MAE: 1129.289807290318
R2 score 85.8000301397728




c:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.049e+07, tolerance: 1.171e+06
  model = cd_fast.enet_coordinate_descent(


In [20]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']